In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch

from src.utils.prepare import prepare_model
from src.modules.early_exit import SDN

In [ ]:
type_names = {
        'model': 'resnet_tunnel',
        'criterion': 'cls',
        'dataset': 'cifar10',
        'optim': 'sgd',
        'scheduler': 'multiplicative'
    }

NUM_CLASSES = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_config = {'backbone_type': 'resnet18',
                    'only_features': False,
                    'batchnorm_layers': True,
                    'width_scale': 1.0,
                    'skips': True,
                    'modify_resnet': True}
model_params = {'model_config': model_config, 'num_classes': NUM_CLASSES, 'dataset_name': type_names['dataset']}

model = prepare_model(type_names['model'], model_params=model_params).to(device)

In [ ]:
sample = torch.randn(16,3,32,32).to(device)
y_true = torch.randint(0, 10, (16,)).to(device)
criterion = torch.nn.CrossEntropyLoss()
model_ee = SDN(model, criterion=criterion, ic_idxs=[0,2,4,5], confidence_threshold=0.5, is_model_frozen=True, prob_conf=True, sample=sample)

In [ ]:
model_ee.run_train(sample, y_true)

In [ ]:
model_ee.run_val(sample, y_true)